In [1]:
import pickle
import numpy as np

# Load your sb3 replay buffer
with open('/home/nitesh/workspace/offline_rl_test/text2nav/IsaacLab/replay_buffer_random_orientation.pkl', 'rb') as f:
    sb3_buffer = pickle.load(f)

In [3]:
len(sb3_buffer)

5000

In [ ]:
rgbs = np.array([item[0] for item in replay_buffer])
goal_indices = np.array([item[1] for item in replay_buffer])
angles = np.array([item[2] for item in replay_buffer])
actions = np.array([item[3] for item in replay_buffer])
rewards = np.array([item[4] for item in replay_buffer])
dones = np.array([item[5] for item in replay_buffer])
truncateds = np.array([item[6] for item in replay_buffer])

In [ ]:
observations = np.array(sb3_buffer[0][:, 0, :])
actions = np.array(sb3_buffer[1][:, 0, :])
rewards = np.array(sb3_buffer[2][:, 0, :])
dones = np.array(sb3_buffer[3][:, 0, :])
truncated = np.array(sb3_buffer[4][:, 0, :])
observations = np.concatenate([observations, rewards], axis=1)

In [ ]:
actions.max(), actions.min()

In [ ]:
observations.shape, rewards.shape

In [ ]:
from d3rlpy.dataset import MDPDataset
import d3rlpy

In [ ]:
mdp_dataset = MDPDataset(observations=observations,
                      actions=actions,
                      rewards=rewards,
                      terminals=dones,
                      timeouts=truncated,
                      action_space=d3rlpy.constants.ActionSpace.CONTINUOUS
                      )


In [ ]:
encoder_factory = d3rlpy.models.VectorEncoderFactory(
    hidden_units=[2000, 2000],
    activation='relu',
)

In [ ]:
from d3rlpy.algos import DQNConfig, SACConfig, IQLConfig, TD3PlusBCConfig, BCConfig, CQLConfig
from d3rlpy.preprocessing import ActionScaler, MinMaxActionScaler

iql = TD3PlusBCConfig(action_scaler=MinMaxActionScaler(), actor_encoder_factory=encoder_factory, critic_encoder_factory=encoder_factory).create(device="cuda:0")
iql.build_with_dataset(mdp_dataset)

In [ ]:
iql.get_action_type()

In [ ]:
from d3rlpy.metrics import TDErrorEvaluator, DiscountedSumOfAdvantageEvaluator

# calculate metrics with training dataset
td_error_evaluator = TDErrorEvaluator(episodes=mdp_dataset.episodes)
discounted_sum_of_advantage_evaluator = DiscountedSumOfAdvantageEvaluator(episodes=mdp_dataset.episodes)

In [ ]:
iql.fit(
    mdp_dataset,
    n_steps=100000,
    save_interval=1000,
)

In [ ]:
iql.save_model('cql_model_main_v2.pt')

In [2]:
import d3rlpy
import torch
from d3rlpy.algos import BCConfig

In [5]:
bc = BCConfig().create(device="cuda:0")
bc.create_impl(observation_shape=(1152,), action_size=2)
# bc.load_model("/home/nitesh/Downloads/bc_model.pt")
model_data = torch.load("/home/nitesh/Downloads/bc_model.pt")

bc._impl.policy.load_state_dict(model_data["imitator"])

Device: cuda:0, type is <class 'str'>


/tmp/ipykernel_1260380/208280584.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load("/home/nitesh/Downloads/bc_model.pt")


<All keys matched successfully>